In [2]:
import wrds
db = wrds.Connection()

db.list_tables(library='optionm_all')

Loading library list...
Done


['borrate1996',
 'borrate1997',
 'borrate1998',
 'borrate1999',
 'borrate2000',
 'borrate2001',
 'borrate2002',
 'borrate2003',
 'borrate2004',
 'borrate2005',
 'borrate2006',
 'borrate2007',
 'borrate2008',
 'borrate2009',
 'borrate2010',
 'borrate2011',
 'borrate2012',
 'borrate2013',
 'borrate2014',
 'borrate2015',
 'borrate2016',
 'borrate2017',
 'borrate2018',
 'borrate2019',
 'borrate2020',
 'borrate2021',
 'borrate2022',
 'borrate2023',
 'distrd',
 'distrprojd1996',
 'distrprojd1997',
 'distrprojd1998',
 'distrprojd1999',
 'distrprojd2000',
 'distrprojd2001',
 'distrprojd2002',
 'distrprojd2003',
 'distrprojd2004',
 'distrprojd2005',
 'distrprojd2006',
 'distrprojd2007',
 'distrprojd2008',
 'distrprojd2009',
 'distrprojd2010',
 'distrprojd2011',
 'distrprojd2012',
 'distrprojd2013',
 'distrprojd2014',
 'distrprojd2015',
 'distrprojd2016',
 'distrprojd2017',
 'distrprojd2018',
 'distrprojd2019',
 'distrprojd2020',
 'distrprojd2021',
 'distrprojd2022',
 'distrprojd2023',
 'exchgd'

In [3]:
db.describe_table(library='optionm_all', table='secnmd')


Approximately 265203 rows in optionm_all.secnmd.


,name,nullable,type,comment
0,secid,True,DOUBLE PRECISION,Security ID
1,effect_date,True,DATE,Effective Date
2,cusip,True,VARCHAR(8),CUSIP Number
3,ticker,True,VARCHAR(6),Ticker Symbol
4,class,True,VARCHAR(1),Class Designator
5,issuer,True,VARCHAR(28),Description of the Issuing Company
6,issue,True,VARCHAR(20),Description of the Issue
7,sic,True,VARCHAR(4),SIC Code


In [4]:
QQQ_secid = db.raw_sql("""
SELECT DISTINCT secid, ticker, issuer
FROM optionm_all.secnmd
WHERE ticker = 'QQQ'
""")
QQQ_secid


,secid,ticker,issuer
0,107899.0,QQQ,INVESCO CAPITAL MANAGEMENT L
1,107899.0,QQQ,NASDAQ 100 TR
2,107899.0,QQQ,POWERSHARES QQQ TRUST


In [11]:
db.describe_table(library='optionm_all', table='opprcd2023')


Approximately 232031728 rows in optionm_all.opprcd2023.


,name,nullable,type,comment
0,secid,True,DOUBLE PRECISION,None
1,date,True,DATE,None
2,symbol,True,VARCHAR(21),None
3,symbol_flag,True,VARCHAR(1),None
4,exdate,True,DATE,None
5,last_date,True,DATE,None
6,cp_flag,True,VARCHAR(1),None
7,strike_price,True,DOUBLE PRECISION,None
8,best_bid,True,DOUBLE PRECISION,None
9,best_offer,True,DOUBLE PRECISION,None


In [14]:
df_opt = db.raw_sql("""
SELECT 
  date,
  exdate,
  cp_flag,
  strike_price,
  delta,
  gamma,
  impl_volatility,
  best_bid,
  best_offer,
  volume,
  open_interest,
  optionid,
  secid
FROM optionm_all.opprcd2023
WHERE 
  secid = 107899
  AND date BETWEEN '2023-01-01' AND '2023-12-31'
  AND delta IS NOT NULL
  AND gamma IS NOT NULL
  AND impl_volatility IS NOT NULL
  AND volume > 0
  AND open_interest > 0
  AND (exdate - date) BETWEEN 25 AND 45

""")
df_opt.tail()


,date,exdate,cp_flag,strike_price,delta,gamma,impl_volatility,best_bid,best_offer,volume,open_interest,optionid,secid
64438,2023-05-01,2023-06-09,P,324000.0,-0.506181,0.022442,0.176600,7.66,7.69,9.0,4.0,154160475.0,107899.0
64439,2023-05-01,2023-06-09,P,325000.0,-0.529266,0.022815,0.174236,8.10,8.13,66.0,109.0,154160476.0,107899.0
64440,2023-05-01,2023-06-09,P,326000.0,-0.552952,0.023129,0.171919,8.56,8.60,34.0,1.0,154160477.0,107899.0
64441,2023-05-01,2023-06-09,P,330000.0,-0.651346,0.023545,0.164026,10.67,10.82,10.0,9.0,154189167.0,107899.0
64442,2023-05-01,2023-06-09,P,333000.0,-0.728984,0.023194,0.157359,12.53,12.71,3.0,3.0,154189170.0,107899.0


In [ ]:
import pandas as pd

years = range(2018, 2024)  # 2023 included

dfs = []

sql_template = """
SELECT 
  date,
  exdate,
  cp_flag,
  strike_price,
  delta,
  gamma,
  impl_volatility,
  best_bid,
  best_offer,
  volume,
  open_interest,
  optionid,
  secid
FROM optionm_all.opprcd{year}
WHERE 
  secid = 107899
  AND date BETWEEN '{start}' AND '{end}'
  AND delta IS NOT NULL
  AND gamma IS NOT NULL
  AND impl_volatility IS NOT NULL
  AND volume > 0
  AND open_interest > 0
  AND (exdate - date) BETWEEN 25 AND 45
"""

for year in years:
    start = f"{year}-01-01"
    end = f"{year}-12-31"
    query = sql_template.format(year=year, start=start, end=end)
    print(f"Extracting {year}...")
    df_year = db.raw_sql(query)
    dfs.append(df_year)

df_opt_all = pd.concat(dfs, ignore_index=True)

df_opt_all.head()


Extracting 2018...
Extracting 2019...
Extracting 2020...
Extracting 2021...
Extracting 2022...
Extracting 2023...


,date,exdate,cp_flag,strike_price,delta,gamma,impl_volatility,best_bid,best_offer,volume,open_interest,optionid,secid
0,2018-01-02,2018-02-02,C,148500.0,0.892498,0.021384,0.186879,10.49,10.70,4.0,40.0,118524041.0,107899.0
1,2018-01-02,2018-02-02,C,149500.0,0.875178,0.024551,0.181047,9.61,9.73,2.0,12.0,118524043.0,107899.0
2,2018-01-02,2018-02-02,C,153000.0,0.791621,0.039376,0.157560,6.48,6.56,3.0,42.0,118524049.0,107899.0
3,2018-01-02,2018-02-02,C,153500.0,0.776244,0.042127,0.153491,6.05,6.11,11.0,98.0,118524050.0,107899.0
4,2018-01-02,2018-02-02,C,154000.0,0.758090,0.044900,0.150400,5.63,5.69,20.0,91.0,118524051.0,107899.0


In [17]:
df_opt_all.to_csv("qqq_option_data_2018_2023.csv", index=False)
